In [2]:
import numpy as np
import pandas as pd
from datetime import date
import datetime as dt
import calendar as calendar

In [39]:
def load_csvs(customerfile, orderfile, verbose):
    customers = pd.read_csv(customerfile)

    if verbose: 
        print(f'{len(customers):,} read from customer file')


    orders = pd.read_csv(orderfile)
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    orders['order_purchase_date'] = orders['order_purchase_timestamp'].dt.date
    orders['order_dow']= orders['order_purchase_timestamp'].dt.strftime(('%A'))

    
    if verbose :
        print((f'{len(orders):,} read from orders file'))

    ordercustsout = pd.merge(customers,orders, on = "customer_id", how = "inner")


    ordercustsout = ordercustsout.sort_values(['customer_unique_id', 'order_purchase_timestamp'])
    
    return ordercustsout

orders_file = 'Data\olist_orders_dataset.csv'
customers_file = 'Data\olist_customers_dataset.csv'



    ##Load data and create DF to last full month, june 2018
custorders_cleaned = load_csvs(customers_file, orders_file, 1)

99,441 read from customer file
99,441 read from orders file


In [41]:
snapshot_date = date(2017, 12, 31)
mask1 = custorders_cleaned['order_purchase_timestamp'].dt.date <= snapshot_date
df_raw = custorders_cleaned[mask1]

df_raw.size



636020

## Attributes 1 3:


In [84]:

## Attribute 1: Number of orders per customer ##
def cust_orders(df): 
    custorders =  df.groupby('customer_unique_id').size().reset_index()
    print("cust_orders completed")
    return custorders


cust_ordersreturned = cust_orders(df_raw)


cust_ordersreturned.size

cust_orders completed


88068

In [83]:
##Attribute 2: 
def get_customer_recency(df):
    cust_recency = df.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
    delta = snapshot_date - cust_recency['order_purchase_timestamp'].dt.date
    cust_recency['order_recency'] = delta.dt.days / 30
    cust_recency['order_recency'] = cust_recency['order_recency'].astype(int)

    cust_recency = cust_recency.drop('order_purchase_timestamp', axis=1)

    return cust_recency


cust_recency = get_customer_recency(df_raw)

cust_recency.size

88068

In [105]:
##Attribute 3: 

df_raw12 = pd.get_dummies(df_raw, columns=['order_dow'])
dummies = [x for x in df_raw12.columns if 'order_dow_' in x]

# aggreagte data to the customer-level, so that we have one record per customer
cust_dow = df_raw12.groupby('customer_unique_id')[dummies].sum().reset_index()

cust_dow.shape



(44034, 8)

In [88]:
cust_dow.shape,cust_recency.shap, cust_ordersreturned.size

((44034, 1), 88068, 88068)

## Attribute 4: Counts of orders by status 

In [81]:
def orderbystatus(df): 

    df_raw = pd.get_dummies(df, columns=['order_status'])
    dummies = [x for x in df_raw.columns if 'order_status' in x]
    count_by_orderstatus = df_raw.groupby('customer_unique_id')[dummies].sum().reset_index()
    return count_by_orderstatus



orderstatuscounts = orderbystatus(df_raw)

orderstatuscounts.size

396306

## Attribute 5: Time between estimated and actual delivery dates

In [92]:

    
df = df_raw

dummies = ['Morning', 'Afternoon', 'Evening']
df['order_purchase_timestamp'] = pd.to_datetime(df.order_purchase_timestamp, format='%Y-%m-%d %H:%M:%S')
df['Late Night'] = df['order_purchase_timestamp'].dt.time.between(dt.time(0), dt.time(5)).astype(int)
df['Morning'] = df['order_purchase_timestamp'].dt.time.between(dt.time(5,0,00,1), dt.time(11)).astype(int)
df['Afternoon'] = df['order_purchase_timestamp'].dt.time.between(dt.time(11,0,00,1), dt.time(16)).astype(int)
df['Evening'] = df['order_purchase_timestamp'].dt.time.between(dt.time(16,0,00,1), dt.time(23, 59, 59, 999999)).astype(int)

PM = df.groupby('customer_unique_id')[dummies].sum().reset_index()

PM.shape



<ipython-input-92-89fced6c8c6d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_purchase_timestamp'] = pd.to_datetime(df.order_purchase_timestamp, format='%Y-%m-%d %H:%M:%S')
<ipython-input-92-89fced6c8c6d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Late Night'] = df['order_purchase_timestamp'].dt.time.between(dt.time(0), dt.time(5)).astype(int)
<ipython-input-92-89fced6c8c6d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

(44034, 4)

In [ ]:
Attribute 6

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  78
order_delivered_carrier_date     1192
order_delivered_customer_date    1732
order_estimated_delivery_date       0
order_purchase_date                 0
order_dow                           0
Morning                             0
Afternoon                           0
Evening                             0
Late Night                          0
dtype: int64

In [95]:
def time_between_estimated_and_actual(df):
    df['order_delivered_customer_date'] = pd.to_datetime(df.order_delivered_customer_date, format='%Y-%m-%d %H:%M:%S')
    df['order_estimated_delivery_date'] = pd.to_datetime(df.order_estimated_delivery_date, format='%Y-%m-%d %H:%M:%S')
    mask = df['order_delivered_customer_date'].dt.date <= snapshot_date  
    df = df[mask]
    
    delta = df['order_delivered_customer_date'].dt.date - df['order_estimated_delivery_date'].dt.date


    df['delta'] = delta.dt.days.astype(int)

    time_between =  df.groupby('customer_unique_id')['delta'].mean().reset_index()
    
    return time_between

timebetween = time_between_estimated_and_actual(df_raw)

timebetween.shape

<ipython-input-95-a509760fcdc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_delivered_customer_date'] = pd.to_datetime(df.order_delivered_customer_date, format='%Y-%m-%d %H:%M:%S')
<ipython-input-95-a509760fcdc8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_estimated_delivery_date'] = pd.to_datetime(df.order_estimated_delivery_date, format='%Y-%m-%d %H:%M:%S')
<ipython-input-95-a509760fcdc8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(39969, 2)

# Merge them 

In [136]:
df_merged = pd.merge(cust_recency, cust_ordersreturned, on='customer_unique_id').merge(cust_dow, on='customer_unique_id').merge(timebetween, on='customer_unique_id').merge(PM, on ='customer_unique_id' ).merge(orderstatuscounts, on='customer_unique_id')

(39969, 22)

### Assign labels (aka the target variable or the dependent variable)


In [132]:
start_date = dt.datetime(2018, 1, 1)
end_date = dt.datetime(2018, 7, 31)

custorders_cleaned.columns

mask= (custorders_cleaned['order_purchase_timestamp'].dt.date >= startdate) & (custorders_cleaned['order_purchase_timestamp'].dt.date < enddate)

target_events_raw = custorders_cleaned[mask]

print (target_events_raw['order_purchase_timestamp'].min(), target_events_raw['order_purchase_timestamp'].max())

2018-01-01 02:48:41 2018-07-31 23:54:20


In [133]:
target_events = target_events_raw.groupby('customer_unique_id').size().reset_index().rename(columns={0: 'purch'})

target_events.head()

,customer_unique_id,purch
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0004bd2a26a76fe21f786e4fbd80607f,1
3,00050ab1314c0e55a6ca13cf7181fecf,1
4,00053a61a98854899e70ed204dd4bafe,1


In [134]:
print(f'Number of customers who made at least one purchase durnig the prediction window: {len(target_events):,}')

Number of customers who made at least one purchase durnig the prediction window: 46,412


In [142]:
df = pd.merge(df_merged, target_events, how="left", on="customer_unique_id")

df.shape

df.head()

df.purch.value_counts()

df['purch'] = [1 if x > 0 else 0 for x in df['purch']]
df.purch.value_counts()


0    39394
1      575
Name: purch, dtype: int64

In [144]:
df.purch.value_counts() / len(df)


df.corr()['purch']

order_recency              -0.012610
0                           0.073325
order_dow_Friday           -0.000696
order_dow_Monday            0.013280
order_dow_Saturday         -0.003070
order_dow_Sunday            0.012385
order_dow_Thursday          0.006427
order_dow_Tuesday           0.005260
order_dow_Wednesday         0.007668
delta                      -0.002853
Morning                     0.014710
Afternoon                   0.006054
Evening                     0.007572
order_status_approved            NaN
order_status_canceled      -0.003255
order_status_created             NaN
order_status_delivered      0.073273
order_status_invoiced      -0.002005
order_status_processing     0.012190
order_status_shipped        0.012351
order_status_unavailable    0.009473
purch                       1.000000
Name: purch, dtype: float64

# Model Building